# Tutorial 2

**In this tutorial you will:**
- learn how to pull a flow from the FlowVerse [Section 1](#1-flowverse)
- Familiarized yourself with the ChatAtomicFlow [Section 2](#2-chatatomicflow)
- Learn how to customize existing flows to your needs [Section 2](#231-customizing-chatatomicflow-creating-a-chatbot-that-answers-in-a-given-language)


In [1]:
%load_ext autoreload
%autoreload 2
#imports
from aiflows.utils.general_helpers import read_yaml_file, quick_load_api_keys
from aiflows.utils import serve_utils
from aiflows.utils import colink_utils
from aiflows.workers import run_dispatch_worker_thread
from aiflows.base_flows import AtomicFlow
from aiflows.messages import FlowMessage
from aiflows import flow_verse
import sys
import os
sys.path.append("..")
from utils import compile_and_writefile, dict_to_yaml
import json
import copy
#Specify path of your flow modules
FLOW_MODULES_PATH = "./"



/Users/nicolasbaldwin/opt/miniconda3/envs/mockenv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#starting a local colink server
cl = colink_utils.start_colink_server()

In [3]:
# Start Worker thread
run_dispatch_worker_thread(cl)

[2024-03-22 07:59:09,214][aiflows.workers.dispatch_worker:220][INFO] - Dispatch worker started in attached thread.
[2024-03-22 07:59:09,224][aiflows.workers.dispatch_worker:221][INFO] - dispatch_point: coflows_dispatch


## 1. FlowVerse

### 1.1 What's the FlowVerse ? 
The FlowVerse is the hub of flows created and shared by our amazing community for everyone to use! These flows are shared on Hugging Face with the intention of being reused by others. Explore our Flows on the FlowVerse [here](https://huggingface.co/aiflows)!


### 1.2. Pulling a Flow from the FlowVerse

To pull the `ChatFlowModule` (check out its card [here](https://huggingface.co/aiflows/ChatFlowModule)) from the FlowVerse, we need to use the `flow_verse.sync_dependencies` function. This function will pull the flow into the current directory. 


In [4]:

from aiflows import flow_verse
# ~~~ Load Flow dependecies from FlowVerse ~~~
dependencies = [
    {"url": "aiflows/ChatFlowModule", "revision": "coflows"},
]

flow_verse.sync_dependencies(dependencies)


[2024-03-22 07:59:10,790][aiflows.flow_verse.loading:775][INFO] - [<interactive>] started to sync flow module dependencies to /Users/nicolasbaldwin/Documents/OneDrive/EPFL/DLAB/aiflow-colink/aiflows/AMLD/ChatFlow/flow_modules...
[2024-03-22 07:59:11,135][aiflows.flow_verse.loading:608][INFO] - aiflows/ChatFlowModule:coflows already synced, skip
[2024-03-22 07:59:11,138][aiflows.flow_verse.loading:825][INFO] - [<interactive>] finished syncing




['/Users/nicolasbaldwin/Documents/OneDrive/EPFL/DLAB/aiflow-colink/aiflows/AMLD/ChatFlow/flow_modules/aiflows/ChatFlowModule']

Let's break this down:
* `dependencies` is a list of dictionaries (in this case, there's only one) indicating which FlowModules we want to pull from the FlowVerse. The dictionary contains two key-value pairs:
  * `url`: Specifies the URL where the flow can be found on Hugging Face. Here, the URL is `aiflows/ChatFlowModule`, where `aiflows` is the name of our organization on Hugging Face (or the username of a user hosting their flow on Hugging Face), and `ChatFlowModule` is the name of the FlowModule containing the `ChatAtomicFlow` on the FlowVerse. Note that the `url` is literally the address of the `ChatFlowModule` on Hugging Face (excluding the https://huggingface.co/). So if you type https://huggingface.co/aiflows/ChatFlowModule in your browser, you will find the Flow.
  * `revision`: Represents the revision id (i.e., the full commit hash) of the commit we want to fetch. Note that if you set `revision` to `main`, it will fetch the latest commit on the main branch.

Note that running the cell above will pull the flow in **flow_modules/aiflows/ChatFlowModule**.

Now that we've fetched the `ChatAtomicFlowModule` from the FlowVerse, we can start creating our own personalized Flow from it.

## 2. ChatAtomicFlow

The ChatAtomicFlow is a minimal wrapper for querying an LLM via an API to generate textuals responses to textual inputs. It employs litellm as a backend to query the LLM via an API. See litellm's supported models and APIs here: https://docs.litellm.ai/docs/providers. In this tutorial, we will be using `openai` as the provider. But you can use any provider supported by litellm.

### 2.1 Inspecting ChatAtomicFlow's Default Configuration

Let's start by inspecting the default configuration of the `ChatAtomicFlow`.

In [5]:
## Print the default configuration
default_cfg = read_yaml_file("flow_modules/aiflows/ChatFlowModule/ChatAtomicFlow.yaml")
print(json.dumps(default_cfg, indent=4))


{
    "_target_": "flow_modules.aiflows.ChatFlowModule.ChatAtomicFlow.instantiate_from_default_config",
    "name": "ChatAtomicFlow",
    "description": "Flow which uses as tool an LLM though an API",
    "enable_cache": true,
    "n_api_retries": 6,
    "wait_time_between_retries": 20,
    "system_name": "system",
    "user_name": "user",
    "assistant_name": "assistant",
    "backend": {
        "_target_": "aiflows.backends.llm_lite.LiteLLMBackend",
        "api_infos": "???",
        "model_name": {
            "openai": "gpt-3.5-turbo"
        },
        "n": 1,
        "max_tokens": 2000,
        "temperature": 0.3,
        "top_p": 0.2,
        "frequency_penalty": 0,
        "presence_penalty": 0,
        "stream": true
    },
    "system_message_prompt_template": {
        "_target_": "aiflows.prompt_template.JinjaPrompt"
    },
    "init_human_message_prompt_template": {
        "_target_": "aiflows.prompt_template.JinjaPrompt",
        "template": "Answer the following ques

For the full list of parameters and their descriptions, please refer to the flow's card [here](https://huggingface.co/aiflows/ChatFlowModule#chatatomicflow-objects) (under configuration parameters). We will be discussing the most important parameters in this tutorial:

- **`input_interface_initialized` and `input_interface_non_initialized`**: These parameters in our configuration specify the keys expected in the input data dictionary when the `ChatAtomicFlow` is called for the first time and subsequently. 
  - `input_interface_non_initialized`: Specifies the keys expected in the input data dictionary when the `ChatAtomicFlow` is called for the first time the flow is called.
  - `input_interface_initialized`: Specifies the keys expected in the input data dictionary after the first call, essentially serving a role similar to the regular `input_interface`. 
The distinction between the two becomes apparent when different inputs are required for the initial query compared to subsequent queries. 
For instance, in ReAct, the first time you query the LLM, the input is provided by a human, such as a question. In subsequent queries, the input comes from the execution of a tool (e.g., a query to Wikipedia). In ReAct's case, these two scenarios are distinguished by `ChatAtomicFlow`'s `input_interface_non_initialized` and `input_interface_initialized` parameters.

-  `backend` is a dictionary containing parameters specific to the LLM. These parameters include:
    - `api_infos`: Your API information (which will be passed later for privacy reasons).
    - `model_name`: A dictionary with key-item pairs, where keys correspond to the `backend_used` attribute of the `ApiInfo` class for the chosen backend, and values represent the desired model for that backend. Model selection depends on the provided `api_infos`. Additional models can be added for different backends, following LiteLLM's naming conventions (refer to LiteLLM's supported providers and model names [here](https://docs.litellm.ai/docs/providers)). For instance, with an Anthropic API key and a desire to use "claude-2," one would check Anthropic's model details [here](https://docs.litellm.ai/docs/providers/anthropic#model-details). As "claude-2" is named the same in LiteLLM, the `model_name` dictionary would be updated as follows:
      ```yaml
      backend:
      _target_: aiflows.backends.llm_lite.LiteLLMBackend
      api_infos: ???
      model_name:
        openai: "gpt-3.5-turbo"
        anthropic: "claude-2"
      ```
    - `n`, `max_tokens`, `top_p`, `frequency_penalty`, `presence_penalty` are generation parameters for LiteLLM's completion function (refer to all possible generation parameters [here](https://docs.litellm.ai/docs/completion/input#input-params-1)).

- `system_message_prompt_template`: This is the system prompt template passed to the LLM.
- `init_human_message_prompt_template`: This is the user prompt template passed to the LLM the first time the flow is called.
- `human_message_prompt_template`: This is the user prompt template passed to the LLM after the first time the flow is called.

All 3 prompts are in Jinja format and contain the following configurable parameter:
  - `template`: The prompt template in Jinja format.
  - `input_variables`: The input variables of the prompt to be passed to the Jinja template (passed when the flow is called).
  - `partial_variables`: The partial variables of the prompt to be passed to the Jinja template (passed directly from the config).



### 2.2 Serving the default Configuration

Now that we have a basic understanding of the `ChatAtomicFlow`'s configuration, let's serve the default configuration of the `ChatAtomicFlow` by calling the `serve` method. This method will return the default configuration of the flow:

In [6]:
serve_utils.serve_flow(
    cl=cl,
    flow_class_name="flow_modules.aiflows.ChatFlowModule.ChatAtomicFlow",
    flow_endpoint="ChatAtomicFlow",
)

[2024-03-22 07:59:11,585][aiflows.utils.serve_utils:116][INFO] - Started serving flow_modules.aiflows.ChatFlowModule.ChatAtomicFlow at flows:ChatAtomicFlow.
[2024-03-22 07:59:11,586][aiflows.utils.serve_utils:117][INFO] - dispatch_point: coflows_dispatch
[2024-03-22 07:59:11,593][aiflows.utils.serve_utils:118][INFO] - parallel_dispatch: False
[2024-03-22 07:59:11,595][aiflows.utils.serve_utils:119][INFO] - singleton: False



True

### 2.3.1 Customizing ChatAtomicFlow: Creating a Chatbot that Answers in a given Language

To showcase how to customize the `ChatAtomicFlow`, we will create a chatbot that answers in a given language. We will use the `ChatAtomicFlow` to create a chatbot that answers in French. The key here is to understand how to get an instance of the `ChatAtomicFlow` with your desired configuration overriding the default configuration.

To do this, we will:
- 1. Copy the default configuration of the `ChatAtomicFlow`
- 2. Override the `system_message_prompt_template` to include the language we want the answer in
- 3. Load our API key in the `api_infos` field of the configuration
- 4. Get an instance of the `ChatAtomicFlow` with our desired configuration overrides
- 5. Run the `ChatAtomicFlow` with our desired configuration !

In [7]:
# STEP 1: Copy the default configuration of the `ChatAtomicFlow`

chatbot_overrides = copy.deepcopy(default_cfg)

# STEP 2: Override the system_message_prompt_template to prompt the user to speak in French
language = "French"
chatbot_overrides["system_message_prompt_template"]["template"] = \
f'You are a helpful chatbot that truthfully answers questions. Answer in the following language: {language}.'

# STEP 3: Load the API keys
from aiflows.backends.api_info import ApiInfo

api_info = [ApiInfo(backend_used="openai", api_key=os.getenv("OPENAI_API_KEY"))]
quick_load_api_keys(chatbot_overrides, api_info)

#STEP 4: Get an instance of the `ChatAtomicFlow` with the overrides
french_chatbot = serve_utils.get_flow_instance(
    cl=cl,
    flow_endpoint="ChatAtomicFlow",
    config_overrides=chatbot_overrides,
)


[2024-03-22 07:59:11,900][aiflows.utils.serve_utils:336][INFO] - Mounted 4e62516b-73f7-43a7-be7b-30af64588fea at flows:ChatAtomicFlow:mounts:local:4e62516b-73f7-43a7-be7b-30af64588fea


In [8]:
# STEP 5: Send messages to the chatbot
input_message1 = FlowMessage(
    data={"id": 0, "query": "What is the capital of Switzerland?"},
)

input_message2 = FlowMessage(
    data={"id": 0, "query": "Where is it located?"},
)

messages = [input_message1, input_message2]

for msg in messages:
    print("~~~~Sent message~~~~\n", msg.data)
    future = french_chatbot.get_reply_future(msg)
    reply = future.get_data()
    print("~~~~Reply~~~~ \n",reply)

~~~~Sent message~~~~
 {'id': 0, 'query': 'What is the capital of Switzerland?'}


[2024-03-22 07:59:12,133][aiflows.workers.dispatch_worker:113][INFO] - 
~~~ Dispatch task ~~~
[2024-03-22 07:59:12,140][aiflows.workers.dispatch_worker:155][INFO] - flow_endpoint: ChatAtomicFlow
[2024-03-22 07:59:12,144][aiflows.workers.dispatch_worker:156][INFO] - flow_id: 4e62516b-73f7-43a7-be7b-30af64588fea
[2024-03-22 07:59:12,149][aiflows.workers.dispatch_worker:157][INFO] - owner_id: local
[2024-03-22 07:59:12,151][aiflows.workers.dispatch_worker:158][INFO] - message_paths: ['push_tasks:8aac5bf0-4972-4361-8e7c-93beb81a88f5:msg']
[2024-03-22 07:59:12,155][aiflows.workers.dispatch_worker:159][INFO] - parallel_dispatch: False

[2024-03-22 07:59:12,255][aiflows.workers.dispatch_worker:182][INFO] - Input message source: Proxy_ChatAtomicFlow
[2024-03-22 07:59:13,327][aiflows.workers.dispatch_worker:113][INFO] - 
~~~ Dispatch task ~~~
[2024-03-22 07:59:13,354][aiflows.workers.dispatch_worker:155][INFO] - flow_endpoint: ChatAtomicFlow
[2024-03-22 07:59:13,356][aiflows.workers.dispatch_wo

~~~~Reply~~~~ 
 {'api_output': 'La capitale de la Suisse est Berne.'}
~~~~Sent message~~~~
 {'id': 0, 'query': 'Where is it located?'}
~~~~Reply~~~~ 
 {'api_output': 'Berne est située dans le centre de la Suisse, sur les rives de la rivière Aar.'}


### 2.3.2 Customizing ChatAtomicFlow: Changing the model

Now we could also change model to use a different language model. For example, we could use the `gpt-4` model to answer in French.

In [9]:
# STEP 1: Copy the default configuration of the `ChatAtomicFlow`
english_chatbot_gpt4_overrides = copy.deepcopy(default_cfg)

# STEP 2: Override the model_name to use GPT-4
english_chatbot_gpt4_overrides["backend"]["model_name"] = {"openai": "gpt-4"}

# STEP 3: Load the API keys
api_info = [ApiInfo(backend_used="openai", api_key=os.getenv("OPENAI_API_KEY"))]
quick_load_api_keys(english_chatbot_gpt4_overrides, api_info)

#STEP 4: Get an instance of the `ChatAtomicFlow` with the overrides
english_chatbot_gpt4 = serve_utils.get_flow_instance(
    cl=cl,
    flow_endpoint="ChatAtomicFlow",
    config_overrides=english_chatbot_gpt4_overrides,
)




[2024-03-22 00:59:17,573][aiflows.utils.serve_utils:336][INFO] - Mounted c9224c21-c2fb-44e4-91dc-487f77fd3659 at flows:ChatAtomicFlow:mounts:local:c9224c21-c2fb-44e4-91dc-487f77fd3659


In [10]:
# STEP 5: Send messages to the chatbot

input_message1 = FlowMessage(
    data={"id": 0, "query": "What is the capital of Switzerland?"},
)

input_message2 = FlowMessage(
    data={"id": 0, "query": "Where is it located?"},
)

messages = [input_message1, input_message2]

for msg in messages:
    print("~~~~Sent message~~~~\n", msg.data)
    future = english_chatbot_gpt4.get_reply_future(msg)
    reply = future.get_data()
    print("~~~~Reply~~~~ \n",reply)

~~~~Sent message~~~~
 {'id': 0, 'query': 'What is the capital of Switzerland?'}


~~~~Reply~~~~ 
 {'api_output': 'The capital of Switzerland is Bern.'}
~~~~Sent message~~~~
 {'id': 0, 'query': 'Where is it located?'}
~~~~Reply~~~~ 
 {'api_output': 'Bern is located in the Swiss Plateau, which is in the central part of Switzerland. It is surrounded by the Aare River.'}


### 2.3.3 Customizing ChatAtomicFlow: Creating an instruction-based chatbot

Now let's write a chatbot that completes your messages by substituting whatever you write in double brackets "[[]]" with the instructions you give it. For example, "the capital of Switzerland is [[insert capital]]" should return "the capital of Switzerland is Bern". This is a simple example of a chatbot that could augment your writing by providing you with the information you need.

In [11]:
# Deepcopy of demo config
overrides_config = copy.deepcopy(default_cfg)

#TODO: Mount a Chatflow who generates text with a `temperature = 1` and a `system_message_prompt_template` that is personalized
overrides_config["backend"]["temperature"] = 1.0
overrides_config["system_message_prompt_template"]["template"] = \
    "You are a helpful chatbot that fills out a given prompt in areas surrounded by brackets '[[]]'. Within the brackets are instructions you should follow."


#loading api key to config
api_info = [ApiInfo(backend_used="openai", api_key=os.getenv("OPENAI_API_KEY"))]
quick_load_api_keys(overrides_config, api_info)


#TO DO get a flow instance
personalized_chatbot = serve_utils.get_flow_instance(
    cl=cl,
    flow_endpoint="ChatAtomicFlow",
    config_overrides=overrides_config,
)

[2024-03-22 00:59:29,523][aiflows.utils.serve_utils:336][INFO] - Mounted a01623d5-1efb-4887-ba65-9e9d06dcc6d7 at flows:ChatAtomicFlow:mounts:local:a01623d5-1efb-4887-ba65-9e9d06dcc6d7


In [12]:
input_message1 = FlowMessage(
    data={"id": 0, "query": "The capital of Switzerland is [[insert capital]]."},
)

query_2 = \
"""
Dear Margaret,
[[Ask about her day and talk about the sunny weather]]

[[Ask if she has finished her part of the report]]. I've finished my part and I'm looking forward to the weekend!

[[Sign of with my name: Nicolas Baldwin]].
"""

input_message2 = FlowMessage(
    data={
        "id": 0,
        "query": query_2
    },
)

messages = [input_message1, input_message2]

for msg in messages:
    print("~~~~Sent message~~~~\n", msg.data)
    future = personalized_chatbot.get_reply_future(msg)
    reply = future.get_data()
    print("~~~~Reply~~~~ \n",reply["api_output"])

~~~~Sent message~~~~
 {'id': 0, 'query': 'The capital of Switzerland is [[insert capital]].'}


~~~~Reply~~~~ 
 The capital of Switzerland is [[Bern]].
~~~~Sent message~~~~
 {'id': 0, 'query': "\nDear Margaret,\n[[Ask about her day and talk about the sunny weather]]\n\n[[Ask if she has finished her part of the report]]. I've finished my part and I'm looking forward to the weekend!\n\n[[Sign of with my name: Nicolas Baldwin]].\n"}
~~~~Reply~~~~ 
 Dear Margaret,

How has your day been? The weather is so sunny and beautiful today!

Have you finished your part of the report yet? I've completed my section and I'm excited for the weekend!

Take care,
Nicolas Baldwin
